In [1]:
import sys
sys.path.insert(1, '/afs/hep.wisc.edu/home/kkboone/software/ssi_corrections_des_y3_balrog/python')
import numpy as np
import fitsio
import healpy as hp
import Config
import StellarConfig as strConfig

In [2]:
pixFile = strConfig.pixFile
deepFiles = Config.deepFiles
deepCols = Config.deepCols

res = strConfig.res

In [3]:
deepCols = np.append(deepCols, np.array(['BDF_MAG_G', 'BDF_MAG_R']))

In [35]:
# Read in valid pixels
validPix = fitsio.read(pixFile)['PIXEL']

# Read in deep field information
deepRA = np.array([])
deepDEC = np.array([])
deepClass = np.array([])
deepFlag = np.array([])
deepFlagNir = np.array([])
deepRmag = np.array([])
deepGmag = np.array([])

for deepFile in deepFiles:
    deepData = fitsio.read(deepFile, columns = deepCols)

    deepRA = np.append(deepRA, deepData['RA'])
    deepDEC = np.append(deepDEC, deepData['DEC'])
    deepClass = np.append(deepClass, deepData['KNN_CLASS'])
    deepFlag = np.append(deepFlag, deepData['MASK_FLAGS'])
    deepFlagNir = np.append(deepFlagNir, deepData['MASK_FLAGS_NIR'])
    deepRmag = np.append(deepRmag, deepData['BDF_MAG_R'])
    deepGmag = np.append(deepGmag, deepData['BDF_MAG_G'])
    
deepGR = deepGmag - deepRmag

# Apply quality cuts and ambiguous class cuts to the deep fields.
deepFlagCuts = np.where((deepFlag == 0) &
                        (deepFlagNir == 0) &
                        (deepRA < 120) &
                        (deepClass > 0) &
                        (deepClass <= 3) &
                        (deepGR >= 0) &
                        (deepGR <= 1) &
                        (deepRmag >= 24.5) & 
                        (deepRmag <= 25))[0]

deepRA = deepRA[deepFlagCuts]
deepDEC = deepDEC[deepFlagCuts]
deepClass = deepClass[deepFlagCuts]

# No class objects with bands missing could present an issue so this is a warning.
if len(np.where(deepClass == 3)[0]) != 0:
    print('WARNING: Objects with no class are present in this deep field selection. ' + str(len(np.where(deepClass == 3)[0])) + ' object(s) out of ' + str(len(deepClass)) + ' have an ambiguous classification.')

# Check which healpixels contain deep field objects to crop wide field objects immediately.
deepPix = hp.ang2pix(res, deepRA, deepDEC, lonlat = True, nest = True)

valPixCut = np.isin(deepPix, validPix)

deepPix = deepPix[valPixCut]
deepClass = deepClass[valPixCut]

In [36]:
ratioStars = len(np.where(deepClass == 2)[0]) / len(np.where((deepClass >= 1) & (deepClass <= 2))[0])
ratioGalas = 1 - ratioStars

In [37]:
starsPerPix = ratioStars * (len(deepPix) / len(np.unique(deepPix)))
galasPerPix = ratioGalas * (len(deepPix) / len(np.unique(deepPix)))

In [38]:
ratioStars

0.006068646961991106

In [39]:
starsPer512 = starsPerPix * 64
galasPer512 = galasPerPix * 64

In [47]:
starsPer512

1.8229482990582355

In [48]:
galasPer512

298.5649817413031